In [1]:
import sys
# sys.path.append(r"C:\Users\David Palecek\Documents\Python_projects\marine_omics_demos\momics-demos")
sys.path.append("/home/david-palecek/coding/emo-bon/momics-demos")

from tables import TaxonomyTable, AbundanceTable
from wfs_extra.scripts.mgnify.mgnify_utils import replace_trailing_empty_with_none, invert_pivot_taxonomic_data
from pathlib import Path
import pandas as pd

%load_ext autoreload
%autoreload 2

## Abundance to taxonomy (MGnify data)

In [2]:
mgnify = AbundanceTable.from_csv(
    Path.home() / "coding/emo-bon/momics-demos/data/mgnify_data/ERP003634_taxonomy_abundances_SSU_v5.0.tsv",
    # Path.home() / "Documents/Python_projects/marine_omics_demos/momics-demos/data/mgnify_data/ERP003634_taxonomy_abundances_SSU_v5.0.tsv",
    sep="\t",
    source="tax_mgnify_raw",
)

INFO:tables.tables:Converting source: tax_mgnify_raw


In [3]:
# mgnify.df.head()

In [4]:
len(mgnify.df), mgnify.df.columns, mgnify.df.columns.name, mgnify.df.index.name, mgnify.source

(497,
 Index(['ERR2010979', 'ERR315850', 'ERR315851', 'ERR315852', 'ERR315853',
        'ERR315854', 'ERR315855'],
       dtype='object', name='source material ID'),
 'source material ID',
 'taxonomic_concat',
 'tax_mgnify_raw')

In [5]:
flag_has_ncbi, taxonomy = mgnify.to_taxonomy_table()
flag_has_ncbi

INFO:tables.tables:self.source before conversion to Taxonomy table tax_mgnify_raw
INFO:tables.tables:Converting source: tax_processed
INFO:tables.sources.converters:Abundance dtype after conversion: uint16
INFO:tables.tables:Validating source: tax_processed


False

In [6]:
taxonomy.df.head()

,abundance,superkingdom,kingdom,phylum,class,order,family,genus,species
source material ID,,,,,,,,,
ERR2010979,4,Archaea,None,None,None,None,None,None,None
ERR2010979,295,Archaea,,Euryarchaeota,Candidatus_Poseidoniia,None,None,None,None
ERR2010979,261,Archaea,,Euryarchaeota,Thermoplasmata,None,None,None,None
ERR2010979,3,Archaea,,Thaumarchaeota,,Nitrosopumilales,Nitrosopumilaceae,None,None
ERR2010979,939,Bacteria,None,None,None,None,None,None,None


## Taxonomy to abundance (EMO-BON data)

In [7]:
import warnings
import os
import holoviews as hv
from skbio.stats.ordination import pcoa
import gc

warnings.filterwarnings('ignore')

import pandas as pd
from momics.utils import load_and_clean

In [8]:
root_folder = os.path.abspath(os.path.join('../../../'))
assets_folder = os.path.join(root_folder, 'assets')

def get_valid_samples():
    df_valid = pd.read_csv(
        os.path.join(root_folder, 'data/shipment_b1b2_181.csv')
    )
    return df_valid

valid_samples = get_valid_samples()

In [9]:
# High level function from the momics.utils module
full_metadata, mgf_parquet_dfs = load_and_clean(valid_samples=valid_samples)

# select categorical columns from metadata
categorical_columns = sorted(full_metadata.select_dtypes(include=['object', "boolean"]).columns)

# select numerical columns from metadata
numerical_columns = sorted(full_metadata.select_dtypes(include=['int64', 'float64']).columns)

In [10]:
ssu= mgf_parquet_dfs['ssu']
ssu.head()

abundance superkingdom kingdom  \
source material ID ncbi_tax_id                                   
EMOBON_BPNS_So_5   2157               7.0      Archaea    None   
                   1801616            1.0      Archaea           
                   28890              1.0      Archaea           
                   183968             1.0      Archaea           
                   192989             3.0      Archaea           

                                                   phylum        class order  \
source material ID ncbi_tax_id                                                 
EMOBON_BPNS_So_5   2157                              None         None  None   
                   1801616      Candidatus_Woesearchaeota         None  None   
                   28890                    Euryarchaeota         None  None   
                   183968                   Euryarchaeota  Thermococci  None   
                   192989                   Nanoarchaeota         None  None   

                               family genus species  
source material ID ncbi_tax_id                       
EMOBON_BPNS_So_5   2157          None  None    None  
                   1801616       None  None    None  
                   28890         None  None    None  
                   183968        None  None    None  
                   192989        None  None    None

In [ ]:
ssu = TaxonomyTable(ssu)
gc.collect()
ssu.source

INFO:tables.tables:Trying to infer source: tax_processed
INFO:tables.tables:Detected source: tax_processed
INFO:tables.sources.converters:Abundance dtype after conversion: uint16


20

In [12]:
ssu.df.head()

abundance superkingdom kingdom  \
source material ID ncbi_tax_id                                   
EMOBON_AAOT_Wa_1   2                 1256     Bacteria    None   
                   29                  20     Bacteria           
                   85                   6     Bacteria           
                   112                  2     Bacteria           
                   126                  1     Bacteria           

                                        phylum                class  \
source material ID ncbi_tax_id                                        
EMOBON_AAOT_Wa_1   2                      None                 None   
                   29           Proteobacteria  Deltaproteobacteria   
                   85           Proteobacteria  Alphaproteobacteria   
                   112          Planctomycetes       Planctomycetia   
                   126          Planctomycetes       Planctomycetia   

                                           order             family  \
source material ID ncbi_tax_id                                        
EMOBON_AAOT_Wa_1   2                        None               None   
                   29               Myxococcales               None   
                   85            Rhodobacterales    Hyphomonadaceae   
                   112          Planctomycetales               None   
                   126          Planctomycetales  Planctomycetaceae   

                                     genus species  
source material ID ncbi_tax_id                      
EMOBON_AAOT_Wa_1   2                  None    None  
                   29                 None    None  
                   85           Hyphomonas    None  
                   112                None    None  
                   126                None    None

### taxonomy -> abundance

In [13]:
abundance_table = ssu.to_abundance_table()
abundance_table.df.head()

INFO:tables.tables:Converting source: abundance_processed
INFO:tables.tables:Validating source: abundance_processed


,source material ID,EMOBON_AAOT_Wa_1,EMOBON_AAOT_Wa_2,EMOBON_AAOT_Wa_22,EMOBON_AAOT_Wa_26,EMOBON_AAOT_Wa_27,EMOBON_AAOT_Wa_41,EMOBON_AAOT_Wa_42,EMOBON_AAOT_Wa_46,EMOBON_AAOT_Wa_47,EMOBON_AAOT_Wa_6,...,EMOBON_VB_Wa_4,EMOBON_VB_Wa_41,EMOBON_VB_Wa_42,EMOBON_VB_Wa_43,EMOBON_VB_Wa_44,EMOBON_VB_Wa_5,EMOBON_VB_Wa_93,EMOBON_VB_Wa_94,EMOBON_VB_Wa_96,EMOBON_VB_Wa_97
ncbi_tax_id,taxonomic_concat,,,,,,,,,,,,,,,,,,,,,
2,2;sk__Bacteria;k__;p__;c__;o__;f__;g__;s__,1256,1783,1920,895,536,848,461,534,304,1172,...,765,769,1106,1449,996,978,456,552,1908,1120
6,6;sk__Bacteria;k__;p__Proteobacteria;c__Alphaproteobacteria;o__Rhizobiales;f__Xanthobacteraceae;g__Azorhizobium;s__,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,10;sk__Bacteria;k__;p__Proteobacteria;c__Gammaproteobacteria;o__Cellvibrionales;f__Cellvibrionaceae;g__Cellvibrio;s__,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,16;sk__Bacteria;k__;p__Proteobacteria;c__Betaproteobacteria;o__Nitrosomonadales;f__Methylophilaceae;g__Methylophilus;s__,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,18;sk__Bacteria;k__;p__Proteobacteria;c__Deltaproteobacteria;o__Desulfuromonadales;f__Desulfuromonadaceae;g__Pelobacter;s__,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### abundance -> taxonomy

In [25]:
 # Convert back to TaxonomyTable
flag_has_ncbi, back_taxonomy_table = abundance_table.to_taxonomy_table()
assert type(back_taxonomy_table).__name__== "TaxonomyTable", "Back conversion should yield a TaxonomyTable instance."
assert back_taxonomy_table.df.equals(ssu.df), "DataFrames should be equal after round-trip conversion."

INFO:tables.tables:DataFrame is abundance with ncbi_tax_id data.
INFO:tables.tables:self.source before conversion to Taxonomy table abundance_processed
INFO:tables.tables:Converting source: tax_processed
INFO:tables.sources.converters:Abundance dtype after conversion: uint16
INFO:tables.tables:Validating source: tax_processed


In [28]:
ssu.df.index.equals(back_taxonomy_table.df.index), ssu.df.columns.equals(back_taxonomy_table.df.columns), ssu.df.equals(back_taxonomy_table.df)

(True, True, True)

In [29]:
[ssu.df[col].equals(back_taxonomy_table.df[col]) for col in ssu.df.columns]

[True, True, True, True, True, True, True, True, True]

In [30]:
print(f"Memory usage: {ssu.df['abundance'].memory_usage(deep=True) / 1024**2:.2f} MB")
print(f"Memory usage: {back_taxonomy_table.df['abundance'].memory_usage(deep=True) / 1024**2:.2f} MB")

Memory usage: 0.96 MB
Memory usage: 0.96 MB


In [31]:
(ssu.df['abundance'].values == back_taxonomy_table.df['abundance'].values).all()

np.True_

In [32]:
test = invert_pivot_taxonomic_data(abundance_table.df, target_col=["taxonomic_concat", "ncbi_tax_id"])
test.head()

abundance superkingdom kingdom  \
source material ID ncbi_tax_id                                   
EMOBON_AAOT_Wa_1   2                 1256     Bacteria    None   
                   29                  20     Bacteria           
                   85                   6     Bacteria           
                   112                  2     Bacteria           
                   126                  1     Bacteria           

                                        phylum                class  \
source material ID ncbi_tax_id                                        
EMOBON_AAOT_Wa_1   2                      None                 None   
                   29           Proteobacteria  Deltaproteobacteria   
                   85           Proteobacteria  Alphaproteobacteria   
                   112          Planctomycetes       Planctomycetia   
                   126          Planctomycetes       Planctomycetia   

                                           order             family  \
source material ID ncbi_tax_id                                        
EMOBON_AAOT_Wa_1   2                        None               None   
                   29               Myxococcales               None   
                   85            Rhodobacterales    Hyphomonadaceae   
                   112          Planctomycetales               None   
                   126          Planctomycetales  Planctomycetaceae   

                                     genus species  
source material ID ncbi_tax_id                      
EMOBON_AAOT_Wa_1   2                  None    None  
                   29                 None    None  
                   85           Hyphomonas    None  
                   112                None    None  
                   126                None    None

In [33]:
assert test.equals(ssu.df), "Inverted data should match the original melted data."